In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Dropout
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns
#from datetime import datetime

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/r3_inframind/datasets/demand_prediction.csv')

In [ ]:
df.head()

,Month,Coarse wool Price,Copra Price,Cotton Price,Fine wool Price,Hard log Price,Hard sawnwood Price,Hide Price,Plywood Price,Rubber Price,Softlog Price,Soft sawnwood Price,Wood pulp Price,production_demand
0,Apr-90,482.34,236,1.83,"1,071.63",161.20,549.91,100.00,312.36,0.84,120.66,218.76,829.29,2042.410
1,May-90,447.26,234,1.89,"1,057.18",172.86,491.88,99.46,350.12,0.85,124.28,213.00,842.51,2017.645
2,Jun-90,440.99,216,1.99,898.24,181.67,495.39,97.90,373.94,0.85,129.45,200.00,831.35,1933.885
3,Jul-90,418.44,205,2.01,895.83,187.96,485.86,96.75,378.48,0.86,124.23,210.05,798.83,1902.150
4,Aug-90,418.44,198,1.79,951.22,186.13,487.52,91.89,364.60,0.88,129.70,208.30,818.74,1928.605


In [ ]:
df.head()

,Month,Coarse wool Price,Copra Price,Cotton Price,Fine wool Price,Hard log Price,Hard sawnwood Price,Hide Price,Plywood Price,Rubber Price,Softlog Price,Soft sawnwood Price,Wood pulp Price,production_demand
0,Apr-90,482.34,236,1.83,"1,071.63",161.20,549.91,100.00,312.36,0.84,120.66,218.76,829.29,2042.410
1,May-90,447.26,234,1.89,"1,057.18",172.86,491.88,99.46,350.12,0.85,124.28,213.00,842.51,2017.645
2,Jun-90,440.99,216,1.99,898.24,181.67,495.39,97.90,373.94,0.85,129.45,200.00,831.35,1933.885
3,Jul-90,418.44,205,2.01,895.83,187.96,485.86,96.75,378.48,0.86,124.23,210.05,798.83,1902.150
4,Aug-90,418.44,198,1.79,951.22,186.13,487.52,91.89,364.60,0.88,129.70,208.30,818.74,1928.605


In [ ]:
df

,Month,Coarse wool Price,Copra Price,Cotton Price,Fine wool Price,Hard log Price,Hard sawnwood Price,Hide Price,Plywood Price,Rubber Price,Softlog Price,Soft sawnwood Price,Wood pulp Price,production_demand
0,Apr-90,482.34,236,1.83,"1,071.63",161.20,549.91,100.00,312.36,0.84,120.66,218.76,829.29,2042.410
1,May-90,447.26,234,1.89,"1,057.18",172.86,491.88,99.46,350.12,0.85,124.28,213.00,842.51,2017.645
2,Jun-90,440.99,216,1.99,898.24,181.67,495.39,97.90,373.94,0.85,129.45,200.00,831.35,1933.885
3,Jul-90,418.44,205,2.01,895.83,187.96,485.86,96.75,378.48,0.86,124.23,210.05,798.83,1902.150
4,Aug-90,418.44,198,1.79,951.22,186.13,487.52,91.89,364.60,0.88,129.70,208.30,818.74,1928.605
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,Dec-19,NaN,NaN,1.67,NaN,272.80,NaN,NaN,500.37,1.66,NaN,NaN,875.00,825.750
357,Jan-20,NaN,NaN,1.74,NaN,272.40,NaN,NaN,499.64,1.68,NaN,NaN,875.00,825.230
358,Feb-20,NaN,NaN,1.69,NaN,270.56,NaN,NaN,496.28,1.61,NaN,NaN,875.00,822.570
359,Mar-20,NaN,NaN,1.49,NaN,276.93,NaN,NaN,507.96,1.50,NaN,NaN,875.00,831.440


In [ ]:
#Variables for training
cols = list(df)[1:25]
# columns 1,2,3,4,5 .. NOT 6
cols

['Coarse wool Price',
 'Copra Price',
 'Cotton Price',
 'Fine wool Price',
 'Hard log Price',
 'Hard sawnwood Price',
 'Hide Price',
 'Plywood Price',
 'Rubber Price',
 'Softlog Price',
 'Soft sawnwood Price',
 'Wood pulp Price',
 'production_demand']

In [ ]:
df = df.dropna()

In [ ]:
df = df.replace(',','', regex=True)


In [ ]:
df = df.replace('%','', regex=True)


In [ ]:
df

,Month,Coarse wool Price,Copra Price,Cotton Price,Fine wool Price,Hard log Price,Hard sawnwood Price,Hide Price,Plywood Price,Rubber Price,Softlog Price,Soft sawnwood Price,Wood pulp Price,production_demand
0,Apr-90,482.34,236,1.83,1071.63,161.20,549.91,100.00,312.36,0.84,120.66,218.76,829.29,2042.410
1,May-90,447.26,234,1.89,1057.18,172.86,491.88,99.46,350.12,0.85,124.28,213.00,842.51,2017.645
2,Jun-90,440.99,216,1.99,898.24,181.67,495.39,97.90,373.94,0.85,129.45,200.00,831.35,1933.885
3,Jul-90,418.44,205,2.01,895.83,187.96,485.86,96.75,378.48,0.86,124.23,210.05,798.83,1902.150
4,Aug-90,418.44,198,1.79,951.22,186.13,487.52,91.89,364.60,0.88,129.70,208.30,818.74,1928.605
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,Feb-17,1029.58,1146.25,1.88,1368.14,263.45,680.49,76.58,483.23,2.71,157.58,287.43,875.00,3186.160
323,Mar-17,1059.60,1016.00,1.91,1454.83,263.48,672.48,77.93,483.27,2.35,160.05,300.42,875.00,3183.660
324,Apr-17,991.12,1044.00,1.92,1404.98,270.34,688.44,75.43,495.87,2.21,159.84,306.60,875.00,3157.875
325,May-17,1019.95,1112.50,1.95,1433.47,265.28,704.52,69.36,486.59,2.10,159.84,306.60,875.00,3218.580


In [ ]:
df_for_training = df[cols].astype(float)
df_for_training.head()

,Coarse wool Price,Copra Price,Cotton Price,Fine wool Price,Hard log Price,Hard sawnwood Price,Hide Price,Plywood Price,Rubber Price,Softlog Price,Soft sawnwood Price,Wood pulp Price,production_demand
0,482.34,236.0,1.83,1071.63,161.20,549.91,100.00,312.36,0.84,120.66,218.76,829.29,2042.410
1,447.26,234.0,1.89,1057.18,172.86,491.88,99.46,350.12,0.85,124.28,213.00,842.51,2017.645
2,440.99,216.0,1.99,898.24,181.67,495.39,97.90,373.94,0.85,129.45,200.00,831.35,1933.885
3,418.44,205.0,2.01,895.83,187.96,485.86,96.75,378.48,0.86,124.23,210.05,798.83,1902.150
4,418.44,198.0,1.79,951.22,186.13,487.52,91.89,364.60,0.88,129.70,208.30,818.74,1928.605


In [ ]:
#LSTM uses sigmoid and tanh that are sensitive to magnitude so values need to be normalized
# normalize the dataset
scaler = StandardScaler()
scaler = scaler.fit(df_for_training)
df_for_training_scaled = scaler.transform(df_for_training)

In [ ]:
#As required for LSTM networks, we require to reshape an input data into n_samples x timesteps x n_features. 
#In this example, the n_features is 2. We will make timesteps = 3. 
#With this, the resultant n_samples is 5 (as the input data has 9 rows).
trainX = []
trainY = []

In [ ]:
n_future = 1   # Number of days we want to predict into the future
n_past = 14     # Number of past days we want to use to predict the future

In [ ]:
for i in range(n_past, len(df_for_training_scaled) - n_future +1):
    trainX.append(df_for_training_scaled[i - n_past:i, 0:df_for_training.shape[1]])
    trainY.append(df_for_training_scaled[i + n_future - 1:i + n_future, 0])

In [ ]:
trainX, trainY = np.array(trainX), np.array(trainY)

In [ ]:
trainX

array([[[-0.48129412, -1.11170232,  0.39218775, ..., -2.1226654 ,
          0.95295927, -0.53051078],
        [-0.59854782, -1.11928689,  0.50498053, ..., -2.29177   ,
          1.03660346, -0.57406839],
        [-0.61950508, -1.18754799,  0.6929685 , ..., -2.67342971,
          0.96599308, -0.72138862],
        ...,
        [-0.9963681 , -1.11170232,  0.44858414, ..., -2.33815634,
         -0.14757551, -1.2755016 ],
        [-0.87075824, -1.15720973,  0.39218775, ..., -2.39922189,
         -0.39325658, -1.25021831],
        [-0.85050289, -1.14962516,  0.37338895, ..., -2.34138577,
         -0.42482878, -1.12729316]],

       [[-0.59854782, -1.11928689,  0.50498053, ..., -2.29177   ,
          1.03660346, -0.57406839],
        [-0.61950508, -1.18754799,  0.6929685 , ..., -2.67342971,
          0.96599308, -0.72138862],
        [-0.69487769, -1.22926311,  0.73056609, ..., -2.3783774 ,
          0.76023598, -0.77720533],
        ...,
        [-0.87075824, -1.15720973,  0.39218775, ..., -

In [ ]:
print('trainX shape == {}.'.format(trainX.shape))
print('trainY shape == {}.'.format(trainY.shape))

trainX shape == (313, 14, 13).
trainY shape == (313, 1).


In [ ]:
df.tail()

,Month,Coarse wool Price,Copra Price,Cotton Price,Fine wool Price,Hard log Price,Hard sawnwood Price,Hide Price,Plywood Price,Rubber Price,Softlog Price,Soft sawnwood Price,Wood pulp Price,production_demand
322,Feb-17,1029.58,1146.25,1.88,1368.14,263.45,680.49,76.58,483.23,2.71,157.58,287.43,875.0,3186.160
323,Mar-17,1059.60,1016.00,1.91,1454.83,263.48,672.48,77.93,483.27,2.35,160.05,300.42,875.0,3183.660
324,Apr-17,991.12,1044.00,1.92,1404.98,270.34,688.44,75.43,495.87,2.21,159.84,306.60,875.0,3157.875
325,May-17,1019.95,1112.50,1.95,1433.47,265.28,704.52,69.36,486.59,2.10,159.84,306.60,875.0,3218.580
326,Jun-17,1065.81,1119.00,1.87,1403.83,268.39,697.44,67.59,492.29,1.72,159.84,306.60,875.0,3229.690


In [ ]:
# forecast_period_dates

In [ ]:
from tensorflow import keras
import pickle

model = keras.models.load_model('/content/drive/MyDrive/r3_inframind/models/production_demand.h5') #lstm

sensor = pickle.load(open('/content/drive/MyDrive/r3_inframind/models/sensor.pkl', 'rb')) #dtree
raw = pickle.load(open('/content/drive/MyDrive/r3_inframind/models/raw_material.pkl', 'rb')) #rf
human = pickle.load(open('/content/drive/MyDrive/r3_inframind/models/human_error.pkl', 'rb')) #rf
defect = pickle.load(open('/content/drive/MyDrive/r3_inframind/models/defect_detection.pkl', 'rb')) #rf


In [ ]:
forecast = model.predict(trainX[-n_future:]) #forecast 

In [ ]:
#Perform inverse transformation to rescale back to original range
#Since we used 5 variables for transform, the inverse expects same dimensions
#Therefore, let us copy our values 5 times and discard them after inverse transform
forecast_copies = np.repeat(forecast, df_for_training.shape[1], axis=-1)
y_pred_future = scaler.inverse_transform(forecast_copies)[:,0]

In [ ]:
# # Convert timestamp to date
# forecast_dates = []
# for time_i in forecast_period_dates:
#     forecast_dates.append(time_i.date())

In [ ]:
y_pred_future

array([855.5643], dtype=float32)

In [ ]:
wt = 0
if(y_pred_future<1100):
  wt += 3

In [ ]:
# # save model
# model.save('/content/drive/MyDrive/r3_inframind/models/production_demand.h5')

In [ ]:
X_raw = [[0.609,	2.13,	4.62,	34.2,	11.12,	4.02,	14],[0.609,	2.13,	4.62,	34.2,	11.12,	4.02,	14]   ]
X_sensor = [[13.7,	7.8,	27,	597.9,	18.1,	49.8,	0.2,	1],[13.7,	7.8,	27,	597.9,	18.1,	49.8,	0.2,	1]            ]
X_human = [[26,	25,	0,	0,	22],[26,	25,	0,	0,	22]]
X_defect =[ [22.9,	15.618,	93.244,	1334.168,	243.131,	351.64,	332.709,		638.672,	6.11,	16.677,	45.512,	70.022,	328.352,	300.954,	183.929,	888.448,	1343.424,	215.372,	54.186,	249.916,	29.01]]



In [ ]:
cf = 0
cf  += raw.predict_proba(X_raw)[:,1][0]
cf += sensor.predict_proba(X_sensor)[:,1][0]
cf += human.predict_proba(X_human)[:,1][0]
cf += defect.predict_proba(X_defect)[:,1][0]

In [ ]:
# cf 

2.635

In [ ]:
if(cf<wt):
  print("NO DEFECTS IN THE BATCH")
else:
  print("DEFECTED BATCH")

NO DEFECTS IN RAW MATERIALS


In [ ]:
# sns.lineplot(original['Date'], original['Open'])
# sns.lineplot(df_forecast['Date'], df_forecast['Open'])